# Chapter2
- 자연어처리 - 컴퓨터가 우리의 말을 이해하게 만드는 것
- 특히 고전적인 방법을 중심으로

## 2.1 자연어 처리란
- 자연어 : 평소에 쓰는 말
- 자연어 처리(NLP) : 자연어를 컴퓨터에게 이해시키기 위한 기술

### 2.1.1 단어의 의미
- 단어 : 의미의 최소단위
- 단어의 의미를 표현하는 방법
    - 시소러스를 활용한 기법
    - 통계기반기법
    - 추론기반기법(word2vec)

## 2.2 시소러스를 이용한 방법
- 시소러스를 이용하여 컴퓨터가 단어의 의미를 이해하도록 하는 것
- 시소러스 : 유의어 사전으로, 뜻이 같은 단어(동의어)/뜻이 비슷한 단어(유의어)가 한 그룹으로 분류되어 있음  
    - 사람이 만듦(사람이 단어의 특성을 직접 추출하는 방법)  
ex) car = auto, automobile, machine, motorcar

- 유의어 집합을 만든 후, 상/하위 or 전체와 부분과 같이 더욱 세세한 관계까지 정의하기도 함
- 상하위관계
    - object
        - motor vehicle(동력차)
            - car
                - suv
                - compact
                - hatch-back
            - go-kart
            - truck
- 이러한 단어들의 관계 그래프인 **단어 네트워크**를 이용하면 컴퓨터에 단어의 의미를 가르칠 수 있음


### 2.2.1 WordNet
- 가장 유명한 시소러스 : WordNet
- WordNet : 유의어, 단어 네트워크를 이용할 수 있음
    - 단어 네트워크를 이용하여 단어

### 2.2.2 시소러스의 문제점
1. 시대변화에 대응하기 어려움
    - 단어는 생성되기도, 소멸하기도 함
    - 시대에 따라 언어의 의미가 변하기도 함
1. 사람을 쓰는 비용이 큼
    - 단어의 수가 굉장히 많고, 이 모든 단어에 대한 관계를 정의하는 것은 어려움
1. 단어의 미묘한 차이를 표현할 수 없음
    - 유의어들을 한 그룹으로 묶기 때문에, 유의어일지라도 미묘한 차이가 있는 것을 표현할 수 없음
        - ex) 빈티지/레트로

## 2.3 통계 기반 기법
- 말뭉치 : 자연어처리를 염두에 두고 수집된 대량의 텍스트 데이터
    - 말뭉치는 사람이 작성한 것이기 때문에 자연어에 대한 사람의 '지식'이 충분히 담겨있음
    - 문장을 쓰는 방법, 단어를 선택하는 방법, 단어의 의미 등 사람이 알고있는 자연어에 대한 지식이 포함되어 있음
- 시소러스와는 다르게 특징을 자동으로 추출하는 것을 목표로 함
- *텍스트 데이터에 대한 추가정보(각 단어의 품사)가 포함되는 경우도 있음*

### 2.3.1 파이썬으로 말뭉치 전처리하기
- 말뭉치를 단어로 분할하고 그 분할된 단어들을 단어 ID목록으로 변환

#### 단어 분할

In [59]:
text = "You say goodbye and I say hello." # 말뭉치

In [52]:
import re # 정규 표현식
text = text.lower()
# 공백을 기준으로 단어 토큰화(단어 분리)
# . 를 고려하여 "." -> " ."로 변환(공백.)
text = text.replace(".", " .")
print(text)

words = re.split("\s+", text) # 공백을 기준으로 분류(공백이 연속되어 있다면 이를 하나로 봄)
# +를 붙이지 않으면 매 공백을 기준으로 분류하게 됨.

you say goodbye and i say hello  .


In [44]:
from nltk.tokenize import RegexpTokenizer
RegexpTokenizer("\s+", gaps=True).tokenize(text)

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

#### 대응하는 ID로 짝지어줌

In [53]:
word_to_id = {} # 단어에서 id로 변환
id_to_word = {} # id에서 단어로 변환

for word in words:
    idx = 0
    if word not in word_to_id.keys(): # id가 아직 부여되지 않은 단어라면
        new_id = len(word_to_id) # 새로 추가될 id
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [55]:
print(word_to_id)
print(id_to_word)

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [57]:
# 문장 전처리 함수
import numpy as np
def preprocess(text):
    text = text.lower() # 소문자로 변환
    text = text.replace("."," .") # 공백을 포함해서 .
    words = re.split("\s+", text) # 공백을 기준으로 분류
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [61]:
corpus, word_to_id, id_to_word = preprocess(text)

### 2.3.2 단어의 분산 표현
- 단어의 분산 표현 : '단어의 의미'를 정확하게 파악할 수 있는 벡터 표현
    - 색에서의 RGB는 3차원 벡터로 색을 정확하게 표현함  
- 단어를 벡터로 표현하는 연구의 기초 : 분포가설
- 방법
    1. 통계 기반 기법 : 어떤 단어에 주목했을 때, 그 주변에 어떤 단어가 몇 번이나 등장하는지를 세어 집계하는 방법

### 2.3.3 분포 가설
- **분포가설 : 단어의 의미는 주변 단어에 의해 형성**
    - 단어 자체에는 의미가 없고, 단어가 사용되는 맥락이 의미를 형성
    - 의미가 같은 단어들은 같은 맥락에서 더 많이 등장
    - ex) I drink wine / i drink beer -> drink는 음료 주변이라는 맥락에서 많이 사용됨
    - ex) i guzzle beer / i guzzle wine -> drink와 비슷한 맥락에서 guzzle이 사용됨 -> 의미가 비슷함을 예상할 수 있음

- 맥락 : 특정 단어를 중심에 둔 그 주변 단어
- 윈도우 크기 : 맥락의 크기(주변 단어를 몇 개나 포함할지)
    - 윈도우 크기 1 -> 좌우 한단어씩
    - 윈도우 크기 2 -> 좌우 두단어씩
    - 상황에 따라서 왼단어만/오른단어만/문장의 시작과 끝을 고려할 수도 있음

### 2.3.4 동시발생 행렬(co-occurrence matrix)
- 통계기반기법에 의해서 형성되는 행렬
- 특정 단어를 중심에 두고 윈도우 크기에 따라서 맥락 형성, 맥락에 어떤 단어가 몇 번이나 등장하는지를 세어 집계하는 방법
    - 특정 단어는 중복 없이 -> word_to_id의 key들

In [64]:
import sys
sys.path.append("..")
import numpy as np
from function.preprocess import preprocess

In [65]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

'You say goodbye and I say hello.'

In [66]:
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


##### you say goodbye and i say hello. -> 말뭉치

- 의미를 파악해야 할 단어 : you, say, goodbye, and, i, hello, .
    - 이 단어의 com -> 6x6 행렬
- 윈도우 크기 : 1
- you 의 맥락의 벡터

|-|you|say|goodbye|and|i|hello|.|
|-|-|-|-|-|-|-|-|
|you|0|1|0|0|0|0|0|

In [68]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [69]:
# 직접 입력
C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,0,1]
])

In [74]:
# 동시발생행렬 구하는 함수
def create_co_matrix(corpus, vocab_size, window_size = 1):
    corpus_size = len(corpus) # 문장의 길이
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32) # 단어의 개수를 통해 com 행렬 형성
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx-i  
            right_idx = idx+i
            
            if left_idx >= 0 : # 왼쪽 경계 확인
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1 # 해당 글자의 왼쪽 문맥 +1
            
            if right_idx < corpus_size: # 오른쪽 경계 확인
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] +=1 # 해당 글자의 오른쪽 문맥 +1
    
    return co_matrix        

In [76]:
C = create_co_matrix(corpus, 7)

In [77]:
print(C[0]) # ID가 0인 단어의 벡터 표현
print(C[4]) # ID가 4인 단어의 벡터 표현
print(C[word_to_id["goodbye"]]) # 단어가 goodbyd인 것의 벡터 표현

[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
[0 1 0 1 0 0 0]


### 2.3.5 벡터 간 유사도
- 벡터 사이의 유사도를 측정하는 방법
    1. 내적
    1. 유클리드 거리
    1. 코사인 유사도
        - 각 벡터를 정규화(벡터의 크기 = 1)한 후 내적한 값
        - 두 벡터가 이루는 각  
        $similarity(x,y) = x.y/||x||||y||$

In [78]:
# 코사인 유사도 구하는 함수
def cos_similarity(x,y, eps=1e-8):
    # 만약 제로 벡터이면 eps 유지 -> 0으로 나누는 것을 막아줌 
    # 만약 제로 벡터가 아니면 eps이 반올림되어 다른 값에 흡수 -> 대부분 최종 결과에 영향 X
    nx = x/np.sqrt(np.sum(x**2)+eps)
    ny = y/np.sqrt(np.sum(y**2)+eps)
    
    return np.dot(nx, ny)

In [1]:
import sys
sys.path.append("..")
from function.cos_similarity import cos_similarity 
from function.create_co_matrix import create_co_matrix
from function.preprocess import preprocess

In [2]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id) # co-matrix의 size
C = create_co_matrix(corpus, vocab_size)

In [3]:
c0 = C[word_to_id["you"]]
c1 = C[word_to_id["i"]]
print(cos_similarity(c0, c1)) # you와 i의 유사도

0.7071067758832467


### 2.3.6 유사 단어의 랭킹 표시
- 어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력하는 함수
- 인자
    - query
    - word_to_id
    - id_to_word
    - word_matrix
    - top=5

In [4]:
# 유사도가 높은 단어를 구하는 함수 
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # 검색어를 꺼냄 
    if query not in word_to_id:
        print("%s(을)를 찾을 수 없습니다." %query)
        return
    
    print("\n[query]"+query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    # 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size) # vocab에 저장되어 있는 각 단어와의 유사도를 담을 배열
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec) # 자기 자신과의 듀사도도 계산되어 있음
    
    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1*similarity).argsort(): # 유사도가 큰 것 부터 index 반환
        if id_to_word[i] == query:
            continue
        print("%s: %s" %(id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return

In [8]:
import numpy as np
x = np.array([100, -20, 2])
print(x.argsort()) # 오름차순 index
print((-x).argsort()) # 내림차순 index

[1 2 0]
[0 2 1]


In [3]:
import sys
sys.path.append("..")
from function.cos_similarity import cos_similarity 
from function.create_co_matrix import create_co_matrix
from function.preprocess import preprocess
from function.most_similar import most_similar

text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar("you", word_to_id, id_to_word, C, top=5)


[query]you
goodbye: 0.7071067758832467
i: 0.7071067758832467
hello: 0.7071067758832467
say: 0.0
and: 0.0
